**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
show_env()

You are working on        Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
REPO DIRECTORY (FD_REPO): /data/reddylab/Kuei/repo 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/work 
DATA DIRECTORY (FD_DATA): /data/reddylab/Kuei/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log 
PROJECT REF     (FD_REF): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/references 



In [2]:
TXT_FOLDER_REGION = "fcc_astarr_macs"

## Import data

```
FD_INP=${FD_DAT}/processed/STARR_ATAC_K562_Reddy_KS91_210401/peaks
FN_INP=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_INP=${FD_INP}/${FN_INP}

cat ${FP_INP} | head
```

```
FD_INP=${FD_DAT}/processed/STARR_ATAC_K562_Reddy_KS91_210401/peaks
FN_INP=KS91_K562_hg38_ASTARRseq_Input.q5.in_all.max_overlaps.bed
FP_INP=${FD_INP}/${FN_INP}

cat ${FP_INP} | head
```

**set the main chromosomes**

In [3]:
txt_fdiry = file.path(FD_DAT, "external")
txt_fname = "chrom.hg38.main.bed"
txt_fpath = file.path(txt_fdiry, txt_fname)

vec = c("Chrom", "ChromStart", "ChromEnd")
dat = read_tsv(txt_fpath, col_names = vec, show_col_types = FALSE)
vec = dat$Chrom

### assign and show
vec_txt_chrom = vec
print(vec)

 [1] "chr1"  "chr10" "chr11" "chr12" "chr13" "chr14" "chr15" "chr16" "chr17"
[10] "chr18" "chr19" "chr2"  "chr20" "chr21" "chr22" "chr3"  "chr4"  "chr5" 
[19] "chr6"  "chr7"  "chr8"  "chr9"  "chrX" 


**Import ASTARR input MACS peaks**

In [4]:
txt_fdiry = file.path(FD_DAT, "processed/STARR_ATAC_K562_Reddy_KS91_210401/peaks")
vec = dir(txt_fdiry)

vec_txt_fname = vec
for(txt in vec){cat(txt, "\n")}

KS91_K562_hg38_ASTARRseq_Input_rep1.masked.dups_marked_peaks.narrowPeak 
KS91_K562_hg38_ASTARRseq_Input_rep2.masked.dups_marked_peaks.narrowPeak 
KS91_K562_hg38_ASTARRseq_Input_rep3.masked.dups_marked_peaks.narrowPeak 
KS91_K562_hg38_ASTARRseq_Input_rep4.masked.dups_marked_peaks.narrowPeak 
KS91_K562_hg38_ASTARRseq_Input_rep5.masked.dups_marked_peaks.narrowPeak 
KS91_K562_hg38_ASTARRseq_Input_rep6.masked.dups_marked_peaks.narrowPeak 
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed 
KS91_K562_hg38_ASTARRseq_Input.q5.in_all.max_overlaps.bed 


In [9]:
vec_txt_fname = c(
    "KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed",
    "KS91_K562_hg38_ASTARRseq_Input.q5.in_all.max_overlaps.bed"
)
names(vec_txt_fname) = c("union", "max_overlaps")
print(vec_txt_fname)

                                                                   union 
"KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed" 
                                                            max_overlaps 
             "KS91_K562_hg38_ASTARRseq_Input.q5.in_all.max_overlaps.bed" 


In [10]:
lst = lapply(vec_txt_fname, function(txt_fname){
    txt_fpath = file.path(txt_fdiry, txt_fname)
    dat = read_tsv(txt_fpath, col_names = FALSE, show_col_types = FALSE)
    return(dat)
})
lst_dat_import = lst

In [11]:
lst = lst_dat_import
head(lst[[1]])

X1,X2,X3
<chr>,<dbl>,<dbl>
chr1,10015,10442
chr1,14253,14645
chr1,16015,16477
chr1,17237,17772
chr1,28903,29613
chr1,30803,31072


In [12]:
lst = lst_dat_import
head(lst[[2]])

X1,X2,X3,X4
<chr>,<dbl>,<dbl>,<dbl>
chr1,10038,10405,6
chr1,14282,14614,6
chr1,16025,16338,6
chr1,17288,17689,6
chr1,28934,29499,6
chr1,115429,115969,6


In [13]:
lst = lst_dat_import
lst = lapply(lst, function(dat){
    dat = dat %>% 
        dplyr::filter(X1 %in% vec_txt_chrom) %>%
        dplyr::select(X1:X3) %>%
        dplyr::mutate(X4 = fun_gen_region(X1, X2, X3)) %>%
        dplyr::arrange(X1, X2, X3)
    return(dat)
})

lst_dat_arrange = lst

**Check filtered results**

In [14]:
lst = lst_dat_import
dat = lst[[1]]
dat = as.data.frame(table(dat$X1))
head(dat)

,Var1,Freq
,<fct>,<int>
1,chr1,30534
2,chr1_KI270706v1_random,35
3,chr1_KI270707v1_random,1
4,chr1_KI270708v1_random,1
5,chr1_KI270709v1_random,5
6,chr1_KI270711v1_random,7


In [15]:
lst = lst_dat_arrange
dat = lst[[1]]
dat = as.data.frame(table(dat$X1))
head(dat)

,Var1,Freq
,<fct>,<int>
1,chr1,30534
2,chr10,11398
3,chr11,12010
4,chr12,10725
5,chr13,3675
6,chr14,4386


**Show results**

In [16]:
lst = lst_dat_arrange
dat = lst[[1]]
fun_display_table(head(dat, 3))

X1,X2,X3,X4
chr1,10015,10442,chr1:10015-10442
chr1,14253,14645,chr1:14253-14645
chr1,16015,16477,chr1:16015-16477


In [17]:
lst = lst_dat_arrange
dat = lst[[2]]
fun_display_table(head(dat, 3))

X1,X2,X3,X4
chr1,10038,10405,chr1:10038-10405
chr1,14282,14614,chr1:14282-14614
chr1,16025,16338,chr1:16025-16338


## Define column description

In [18]:
### set column name and description
dat = tribble(
    ~Name,        ~Note,
    "Chrom",      "Name of the chromosome",
    "ChromStart", "The starting position of the feature in the chromosome",
    "ChromEnd",   "The ending position of the feature in the chromosome",
    "Region",     "chr:start-end for each row"
)

### assign and show
dat_cname = dat
fun_display_table(dat)

Name,Note
Chrom,Name of the chromosome
ChromStart,The starting position of the feature in the chromosome
ChromEnd,The ending position of the feature in the chromosome
Region,chr:start-end for each row


## Save results

In [19]:
txt_folder = TXT_FOLDER_REGION
txt_fdiry  = file.path(FD_RES, "region", txt_folder, "summary")
txt_fname  = "description.tsv"
txt_fpath  = file.path(txt_fdiry, txt_fname)

dir.create(txt_fdiry, showWarnings = FALSE)
dat = dat_cname
write_tsv(dat, txt_fpath)

In [21]:
lst = lst_dat_arrange
for (idx in names(lst)){
    txt_fname = paste(
        "K562.hg38.ASTARR.macs.KS91.input.rep_all",
        idx,
        "q5.bed.gz",
        sep = "."
    )
    print(idx)
    print(txt_fname)
}

[1] "union"
[1] "K562.hg38.ASTARR.macs.KS91.input.rep_all.union.q5.bed.gz"
[1] "max_overlaps"
[1] "K562.hg38.ASTARR.macs.KS91.input.rep_all.max_overlaps.q5.bed.gz"


In [22]:
### set file path
txt_folder = TXT_FOLDER_REGION
txt_fdiry  = file.path(FD_RES, "region", txt_folder)
dir.create(txt_fdiry, showWarnings = FALSE)

lst = lst_dat_arrange
for (idx in names(lst)){
    txt_fname = paste(
        "K562.hg38.ASTARR.macs.KS91.input.rep_all",
        idx,
        "q5.bed.gz",
        sep = "."
    )
    txt_fpath = file.path(txt_fdiry, txt_fname)
    
    dat = lst[[idx]]
    write_tsv(dat, txt_fpath, col_names = FALSE)
    
    print(idx)
    print(txt_fname)
}

[1] "union"
[1] "K562.hg38.ASTARR.macs.KS91.input.rep_all.union.q5.bed.gz"
[1] "max_overlaps"
[1] "K562.hg38.ASTARR.macs.KS91.input.rep_all.max_overlaps.q5.bed.gz"
